# Task 2:Classical GNN

Graph Neural Network are used to handle the data in the form of vertices and edges unline CNN,RNN etc.
It helps to establish relationship between entities

We will first start by importing all the required libraries and will be using  **pytorch** for implementation

In [1]:
import os
import glob
import numpy as np
import torch
import torch.nn.functional as F

from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_cluster import knn_graph
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import torch.optim as optim
from torch_geometric.nn import GCNConv, GATConv, GINConv, BatchNorm, global_mean_pool
from torch.nn import BatchNorm1d

## Data Preprocessing

We will create a class for QGDataset.

The dataset is in .npz files.
Since there are many files we will join all the files and then store the processed data in data.pt 

The dataset has 2 labels X and Y 
### X field
X contain a 2D matrix that is collection of particles and each particles has 4 field 

X.shape= (number of jets,number of particle,features)


The number of jets are the total jets in the data set


The number of particles is total particles in each jet

The features =4 ie pT, eta, phi and PDG ID

### Y field
Y.shape= (num of jets)

It is a 1D matrix that contain only 0 and 1 for Gluon and Quark 

For X we are using float32 as datatype because of memory constraint on the system

We will be using only first 3 features of X ie pT, phi and eta and we will not use PDGID beacuse it is just the id of particle and it does no define any spatial behaviour 

### Graph

We will be using KNN to create a graph

Node/Vertices= Particle in a Jet

Edges= Relationship between particles based on pT, phi, eta

In [ ]:
class QGDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(QGDataset, self).__init__(root, transform, pre_transform)
        path = self.processed_paths[0]
        if os.path.exists(path):
            self.data, self.slices = torch.load(path)
        else:
            self.process()
            self.data, self.slices = torch.load(path)

    @property
    def raw_file_names(self):
        # All .npz files in the raw directory
        return glob.glob(os.path.join(self.raw_dir, '*.npz'))

    @property
    def processed_file_names(self):
        return ['processed_data.pt']

    def process(self):
        data_list = []
        for fpath in tqdm(self.raw_file_names, desc="Processing .npz Files"):
            npz = np.load(fpath, mmap_mode='r')
            # Use the keys 'X' and 'y' as found in your files
            features = npz['X'].astype(np.float32)  # shape: (num_jets, num_particles, num_features)
            labels = npz['y'].astype(np.int64)      # shape: (num_jets,)
            num_jets, num_particles, num_feats = features.shape

            for i in range(num_jets):
                x_np = features[i]  # shape: (num_particles, num_feats)
                mean = np.mean(x_np, axis=0, keepdims=True)
                std = np.std(x_np, axis=0, keepdims=True) + 1e-6
                x_np_norm = (x_np - mean) / std

                x = torch.tensor(x_np_norm, dtype=torch.float32)
                y = torch.tensor([labels[i]], dtype=torch.long)

                # Build graph: use first three features (pT, η, ϕ) for KNN
                coords = x[:, :3]
                edge_index = knn_graph(coords, k=10, loop=False)
                data_obj = Data(x=x, edge_index=edge_index, y=y)
                data_list.append(data_obj)

        self.data, self.slices = self.collate(data_list)
        torch.save((self.data, self.slices), self.processed_paths[0])

## First Model : Simple GCN

### 1. Model Components  
<code>def __init__(self, in_channels, hidden_dim=64, out_dim=2):</code>  
The `in_channels` is the number of input features per node = 3 (`pT, η, ϕ`).<br>  
`hidden_dim=64`: Hidden layer size.<br>  
`out_dim = 2`: We have only 2 outputs (0 for Gluon, 1 for Quark).<br>  

---

### 2. Layers in the Model  
<code>self.conv1 = GCNConv(in_channels, hidden_dim)</code>  
Applies the first Graph Convolutional Layer, which extracts meaningful features from input nodes.<br>  

<code>self.conv2 = GCNConv(hidden_dim, hidden_dim)</code>  
A second Graph Convolutional Layer for refining learned representations.<br>  

<code>self.lin = torch.nn.Linear(hidden_dim, out_dim)</code>  
A fully connected layer that converts the graph representation into logits for classification.<br>  

---

### 3. Forward Pass  
<code>def forward(self, x, edge_index, batch):</code>  
Defines how input data passes through the model.<br>  

#### 1st GCN Layer- Non-Linearity (ReLU)  
<code>x = self.conv1(x, edge_index)</code><br>  
Applies the first graph convolution (message passing).<br>  
<code>x = F.relu(x)</code><br>  
Uses ReLU activation to introduce non-linearity.<br>  

#### 2nd GCN Layer - Non-Linearity (ReLU)  
<code>x = self.conv2(x, edge_index)</code><br>  
Refines the node features based on graph connections.<br>  
<code>x = F.relu(x)</code><br>  
Applies another non-linearity.<br>  

#### Global Pooling  
<code>x = global_mean_pool(x, batch)</code><br>  
Aggregates all node features to form a single graph-level feature vector.<br>  

#### Fully Connected Output  
<code>x = self.lin(x)</code><br>  
Final classification layer for binary output (Quark vs Gluon).<br>  
<code>return x</code><br>  
Returns logits for classification.<br>  

---

In [3]:
class SimpleGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_dim=64, out_dim=2):
        super(SimpleGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.lin = torch.nn.Linear(hidden_dim, out_dim)
    
    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        return self.lin(x)

## Second Model: GCNModel

### 1. Model Components  
<code>def __init__(self, in_channels, hidden_dim=256, out_dim=2, dropout=0.3):</code>  
- `in_channels`: Number of input features per node = 3 (`pT, η, ϕ`).  
- `hidden_dim=256`: Size of the hidden layers for feature extraction.  
- `out_dim=2`: We have only 2 output classes (0 for Gluon, 1 for Quark).  
- `dropout=0.3`: Probability of randomly dropping neurons to prevent overfitting.  

---

### 2. Layers in the Model  
<code>self.conv1 = GCNConv(in_channels, hidden_dim)</code>  
First Graph Convolutional Layer to process input node features.<br>  

<code>self.bn1 = BatchNorm(hidden_dim)</code>  
Batch Normalization to stabilize training and speed up convergence.<br>  

<code>self.conv2 = GCNConv(hidden_dim, hidden_dim)</code>  
Second Graph Convolutional Layer for deeper feature learning.<br>  

<code>self.bn2 = BatchNorm(hidden_dim)</code>  
Another Batch Normalization for improved stability.<br>  

<code>self.dropout = dropout</code>  
Dropout to prevent overfitting during training.<br>  

<code>self.lin = torch.nn.Linear(hidden_dim, out_dim)</code>  
Fully connected layer for final classification.<br>  

---

### 3. Forward Pass  
<code>def forward(self, x, edge_index, batch):</code>  
Defines how data moves through the model.<br>  

#### 1st GCN Layer → BatchNorm → ReLU → Dropout  
<code>x = self.conv1(x, edge_index)</code>  
Applies the first graph convolution layer.<br>  

<code>x = self.bn1(x)</code>  
Normalizes feature distribution.<br>  

<code>x = F.relu(x)</code>  
Applies ReLU activation for non-linearity.<br>  

<code>x = F.dropout(x, p=self.dropout, training=self.training)</code>  
Randomly drops some neurons for regularization.<br>  

#### 2nd GCN Layer → BatchNorm → ReLU  
<code>x = self.conv2(x, edge_index)</code>  
Extracts higher-level features from the graph.<br>  

<code>x = self.bn2(x)</code>  
Batch normalization to maintain stable distributions.<br>  

<code>x = F.relu(x)</code>  
Applies another ReLU activation.<br>  

#### Global Pooling  
<code>x = global_mean_pool(x, batch)</code>  
Aggregates features across all nodes in the graph.<br>  

#### Fully Connected Output  
<code>x = self.lin(x)</code>  
Final classification layer for predicting Quark (1) or Gluon (0).<br>  

<code>return x</code>  
Returns the logits for classification.<br>  

---  


In [4]:
class GCNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_dim=256, out_dim=2, dropout=0.3):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_dim)
        self.bn1 = BatchNorm1d(hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.bn2 = BatchNorm1d(hidden_dim)
        self.dropout = dropout
        self.lin = torch.nn.Linear(hidden_dim, out_dim)
    
    def forward(self, x, edge_index, batch):
        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.relu(self.bn2(self.conv2(x, edge_index)))
        x = global_mean_pool(x, batch)
        return self.lin(x)

## Third Model : GATModel 

### 1. Model Components  
<code>def __init__(self, in_channels, hidden_dim=256, out_dim=2, heads=4, dropout=0.3):</code>  
- `in_channels`: Number of input features per node = 3 (`pT, η, ϕ`).  
- `hidden_dim=256`: Size of the hidden layers for feature extraction.  
- `out_dim=2`: We have only 2 output classes (0 for Gluon, 1 for Quark).  
- `heads=4`: Number of attention heads in the first GAT layer.  
- `dropout=0.3`: Dropout rate to prevent overfitting.  

---

### 2. Layers in the Model  
<code>self.conv1 = GATConv(in_channels, hidden_dim, heads=heads, concat=True)</code>  
First Graph Attention Layer (GAT) with multi-head attention (concatenated output).<br>  

<code>self.bn1 = BatchNorm(hidden_dim * heads)</code>  
Batch Normalization to stabilize training and speed up convergence.<br>  

<code>self.conv2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, concat=False)</code>  
Second GAT Layer (single-head) to refine learned features.<br>  

<code>self.bn2 = BatchNorm(hidden_dim)</code>  
Another Batch Normalization for improved stability.<br>  

<code>self.dropout = dropout</code>  
Dropout to prevent overfitting during training.<br>  

<code>self.lin = torch.nn.Linear(hidden_dim, out_dim)</code>  
Fully connected layer for final classification.<br>  

---

### 3. Forward Pass  
<code>def forward(self, x, edge_index, batch):</code>  
Defines how data moves through the model.<br>  

#### 1st GAT Layer → BatchNorm → ELU → Dropout  
<code>x = self.conv1(x, edge_index)</code>  
Applies the first graph attention convolution.<br>  

<code>x = self.bn1(x)</code>  
Normalizes feature distribution.<br>  

<code>x = F.elu(x)</code>  
Applies ELU activation for smooth feature transformation.<br>  

<code>x = F.dropout(x, p=self.dropout, training=self.training)</code>  
Randomly drops some neurons for regularization.<br>  

#### 2nd GAT Layer → BatchNorm → ELU  
<code>x = self.conv2(x, edge_index)</code>  
Extracts refined features from the graph.<br>  

<code>x = self.bn2(x)</code>  
Batch normalization to maintain stable distributions.<br>  

<code>x = F.elu(x)</code>  
Applies another ELU activation.<br>  

#### Global Pooling  
<code>x = global_mean_pool(x, batch)</code>  
Aggregates features across all nodes in the graph.<br>  

#### Fully Connected Output  
<code>x = self.lin(x)</code>  
Final classification layer for predicting Quark (1) or Gluon (0).<br>  

<code>return x</code>  
Returns the logits for classification.<br>  

---


In [5]:
class GATModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_dim=256, out_dim=2, heads=4, dropout=0.3):
        super(GATModel, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_dim, heads=heads, concat=True)
        self.bn1 = BatchNorm1d(hidden_dim * heads)
        self.conv2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, concat=False)
        self.bn2 = BatchNorm1d(hidden_dim)
        self.dropout = dropout
        self.lin = torch.nn.Linear(hidden_dim, out_dim)
    
    def forward(self, x, edge_index, batch):
        x = F.elu(self.bn1(self.conv1(x, edge_index)))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.elu(self.bn2(self.conv2(x, edge_index)))
        x = global_mean_pool(x, batch)
        return self.lin(x)

# Fourth Model: Residual GCN

### 1. Model Components
<code>def __init__(self, in_channels, hidden_dim=256, out_dim=2, dropout=0.3):</code>
- **in_channels**: Number of input features per node (e.g., 3 for pT, η, ϕ).  
- **hidden_dim = 256**: Size of the hidden layers used for feature extraction.  
- **out_dim = 2**: Number of output classes (e.g., 0 for Gluon, 1 for Quark).  
- **dropout = 0.3**: Dropout rate to help prevent overfitting.

---

### 2. Layers in the Model
- **<code>self.conv1 = GCNConv(in_channels, hidden_dim)</code>**  
  The first graph convolutional layer, which transforms input features into a hidden representation.

- **<code>self.bn1 = BatchNorm1d(hidden_dim)</code>**  
  Batch normalization for the output of the first GCN layer to stabilize and speed up training.

- **<code>self.conv2 = GCNConv(hidden_dim, hidden_dim)</code>**  
  The second GCN layer that further processes the features.

- **<code>self.bn2 = BatchNorm1d(hidden_dim)</code>**  
  Batch normalization applied after the second GCN layer.

- **<code>self.conv3 = GCNConv(hidden_dim, hidden_dim)</code>**  
  The third GCN layer which extracts deeper features from the graph.

- **<code>self.bn3 = BatchNorm1d(hidden_dim)</code>**  
  Batch normalization applied to the output of the third GCN layer.

- **<code>self.lin = torch.nn.Linear(hidden_dim, out_dim)</code>**  
  A fully connected layer that maps the final hidden representation to the output classes.

---

### 3. Forward Pass
<code>def forward(self, x, edge_index, batch):</code>

1. **First GCN Layer:**  
   <code>x1 = F.relu(self.bn1(self.conv1(x, edge_index)))</code>  
   - Applies the first GCN layer, followed by batch normalization and ReLU activation.

2. **Second GCN Layer:**  
   <code>x2 = F.relu(self.bn2(self.conv2(x1, edge_index)))</code>  
   - Applies the second GCN layer on the output of the first layer, followed by batch normalization and ReLU activation.

3. **Third GCN Layer with Residual Connection:**  
   <code>x3 = F.relu(self.bn3(self.conv3(x2, edge_index)) + x1)</code>  
   - The output of the third GCN layer is added to the output of the first layer (residual connection) to help preserve initial features, then batch normalization and ReLU activation are applied.

4. **Global Pooling:**  
   <code>x = global_mean_pool(x3, batch)</code>  
   - Aggregates node features into a single graph-level representation by taking the mean of all node features.

5. **Final Classification:**  
   <code>x = self.lin(x)</code>  
   - The pooled features are passed through a linear layer to produce the final logits for classification.

6. **Return:**  
   <code>return x</code>  
   - Returns the output logits.

---



In [6]:
class ResidualGCNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_dim=256, out_dim=2, dropout=0.3):
        super(ResidualGCNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_dim)
        self.bn1 = BatchNorm1d(hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.bn2 = BatchNorm1d(hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.bn3 = BatchNorm1d(hidden_dim)
        self.dropout = dropout
        self.lin = torch.nn.Linear(hidden_dim, out_dim)

    def forward(self, x, edge_index, batch):
        x1 = F.relu(self.bn1(self.conv1(x, edge_index)))
        x2 = F.relu(self.bn2(self.conv2(x1, edge_index)))
        # Residual connection: add output of first layer (x1) to third layer's output
        x3 = F.relu(self.bn3(self.conv3(x2, edge_index)) + x1)
        x = global_mean_pool(x3, batch)
        x = self.lin(x)
        return x

## Training 

### 1. Cross-Entropy Loss 
(`F.cross_entropy(out, data.y)`)<br>
It measures how bad model predictions are.

- The model gives a **score** for each class (e.g., **quark** or **gluon**), but these scores might not be correct.  
- The **cross-entropy loss** tells us **how far off** the model’s predictions are from the actual answers.  
- The goal is to **reduce this loss** so the model makes better predictions over time.  

**Example:**  
If the actual label is **1 (quark)** and the model predicts **[0.1, 0.9]**, meaning it's **90% sure it’s a quark**, the loss is **small** (good!).  
If the model predicts **[0.9, 0.1]**, meaning it's **90% sure it’s a gluon**, the loss is **large** (bad!).  

---

### 2. Optimizer 
(`optimizer.step()`)<br>
- It helps to change parameters after mistakes
- It looks at the **cross-entropy loss** and decides how to **adjust the model’s settings (weights)** to improve its predictions.  
- Popular optimizers:  
  - **SGD (Slow Learner)** – Adjusts weights little by little.  
  - **Adam (Smart Learner)** – Learns **faster** by adjusting weights differently for each part of the model.  

---

### 3. `loss.backward()`
- This tells the model **which parts of itself are responsible for the mistake** (computing gradients).  
- It helps figure out which **weights** should be changed and by how much.  

---

### 4. `model.train()` 
- **`model.train()`** – The model is **actively learning**. It allows techniques like **dropout** (randomly ignoring some neurons) to **prevent overfitting**.
### 5. `model.eval()`   
- **`model.eval()`** – The model is in **testing mode**, meaning no tricks like dropout—it just makes predictions **as accurately as possible**.  

---

## How It All Works Together  
1. The model makes a **guess** (prediction).  
2. The **cross-entropy loss** checks how wrong the guess is.  
3. **Backpropagation (`loss.backward()`)** finds out **what to fix**.  
4. The **optimizer (`optimizer.step()`)** updates the model to **reduce errors** next time.  
5. Over time, the model **gets better at classifying quarks and gluons!**  


In [7]:
def train(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    for data in tqdm(loader, desc="Training"):
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = F.cross_entropy(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(loader.dataset)

def test(model, loader, device):
    model.eval()
    y_true, y_pred = [], []
    for data in loader:
        data = data.to(device)
        with torch.no_grad():
            out = model(data.x, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            y_true.extend(data.y.cpu().numpy())
            y_pred.extend(pred.cpu().numpy())
    return accuracy_score(y_true, y_pred)

In [9]:
def main():
    root_dir = 'qg_data'
    dataset = QGDataset(root=root_dir)
    train_size = int(0.8 * len(dataset))
    train_dataset, test_dataset = dataset[:train_size], dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    in_channels = dataset[0].x.shape[1]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    models = {
        "SimpleGCN": SimpleGCN(in_channels),
        "GCNModel": GCNModel(in_channels),
        "GATModel": GATModel(in_channels),
        "ResidualGCNModel": ResidualGCNModel(in_channels)
    }
    
    for model_name, model in models.items():
        print(f"\nTraining {model_name}...")
        model.to(device)
        optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
        best_acc=0
        for epoch in range(1, 21):
            loss = train(model, train_loader, optimizer, device)
            acc = test(model, test_loader, device)
            if(acc>best_acc):
                best_acc=acc
            print(f"Epoch {epoch:02d}, Loss: {loss:.4f}, Test Accuracy: {acc:.4f}")
        print(f"Best accuracy {best_acc}")

if __name__ == "__main__":
    main()


C:\Users\arnav\AppData\Local\Temp\ipykernel_1468\2633872568.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(path)



Training SimpleGCN...


Training: 100%|██████████| 125/125 [00:01<00:00, 87.22it/s] 


Epoch 01, Loss: 0.6852, Test Accuracy: 0.6830


Training: 100%|██████████| 125/125 [00:00<00:00, 162.12it/s]


...

Epoch 18, Loss: 0.5148, Test Accuracy: 0.7650


Training: 100%|██████████| 125/125 [00:02<00:00, 54.72it/s]


Epoch 19, Loss: 0.5171, Test Accuracy: 0.7840


Training: 100%|██████████| 125/125 [00:02<00:00, 54.68it/s]


Epoch 20, Loss: 0.5157, Test Accuracy: 0.7730
Best accuracy 0.784


# Accuracy 

Simple GCN ----76.5%<br>
GCN Model ----78.1%<br>
GAT Model ----77.7%<br>
Residual GCN ----78.4%

### Simple GCN
It is a simple model with only 2 layers and it cannot capture complex relations

### GCN Model
The GCN model adds batch normalization that helps to stabalize the learning rates

### GAT Model
Graph Attention Network applies attention mechanism that weigh the importance of neighbour nodes

### Residual GCN
It adds recidual connection which helps to preserve low level features and improves gradient flow

# Future Improvements

We can do hyperparameter tuning and increase the epochs and try new and complex models that can increase the accuracy 

Due to memory and computational issues I have to preprocess the data in batch and we have to convert float64 to float32 
I have implemented four simple models that take less computation as fine tuning took a lot of time and increasing layer or hidden dimension had both time and memory issues, so I implemented 4 models.